In the original paper of XG_Boost, the authors consider regression problems solving by tree ensembles [[1]](#1).
$$
\hat{y}_{i}=\sum_{k=1}^{K}f_{k}(\boldsymbol{x}_i)
$$
where $i$ runs between $1$ to $N$ is the index for the unit in a sample and $N$ is the sample size. The sample can be represented as $\mathcal{D}=\{(\boldsymbol{x}_i,{y}_{i})|\boldsymbol{x}_i\in R^{m},{y}_{i} \in R\}$.

The ideas behind the traning of XG_Boost has two folds:
* add decision tree sequentially to minimize the residual loss function of the current tree ensemble, i.e., correct previous trees' mistake.
* try to avoid over fitting by adding regularization term to punish complicated tree structure and one tree dominating the prediction

If the prediction $\hat{y}$ has even contribution from each tree, then misprediction or overfitting can happen only the majority of tree in the ensemble is overfitting or make wrong prediction which has a much lower possibility compared to the single decision tree model. 

Thus the loss function can be designed as following:
$$
\mathcal{L}=\frac{1}{N}\sum_{i=1}^{N}l(\hat{y}_i,y_i)+\frac{1}{K}\sum_{k=1}^{K} \Omega(f_k),\\
\textrm{where} \; \Omega(f_k)= \gamma T + \frac{1}{2}\lambda \sum_{t=1}^{T}\omega_{t}^{2}.
$$The first term is the loss due to difference between prediction and target, which has a minimal when they are equal. The second term is a regularization to decision tree. 

A decision tree $f$ is determined by its tree structure $q$ including decisions on each node and leaf weights $\boldsymbol{\omega}=(\omega_1,\omega_2,...,\omega_T)\in R^{T}$. $q$ will assign $\boldsymbol{x}_i$ a leaf $q(\boldsymbol{x}_i)=1,2,...,T$ and the prediction will be the weight on that leaf $\omega_{q(\boldsymbol{x}_i)}$. So we have
$$
f(\boldsymbol{x}_i)=\omega_{q(\boldsymbol{x}_i)}
$$The regularization term focuses on the number of leaves $T$ and weights $\omega$.

To minimize the total loss function, an approximate method is to add regression tree one by one each minimize the part of loss function that is related to current tree structure. Suppose we have $K$ trees right now, we want to add $(K+1)$th tree to further minimize the total loss function. This will change both terms as:
$$
\mathcal{L}=\frac{1}{N}\sum_{i=1}^{N}l(\hat{y}_i+f_{k+1}(\boldsymbol{x}_i),y_i)+\frac{1}{K+1}\sum_{k=1}^{K+1} \Omega(f_k).
$$

In case the loss function only depends on the difference between $\hat{y}$ and $y$, the above expression can be also written as:
$$
\mathcal{L}=\frac{1}{N}\sum_{i=1}^{N}l(f_{k+1}(\boldsymbol{x}_i)-(y_i-\hat{y}_i))+\frac{1}{K+1}\sum_{k=1}^{K+1} \Omega(f_k).
$$
The part that is related to the $K+1$th tree reads:
$$
\Delta\mathcal{L}_{K+1}=\frac{1}{N}\sum_{i=1}^{N}l(f_{k+1}(\boldsymbol{x}_i)-(y_i-\hat{y}_i))+\frac{1}{K+1} \Omega(f_{k+1})
$$

So for the $K+1$th tree there is no difference in training from a normal regression tree.

**Taylor Expansion**

To isolate the contribution of the $(K+1)$th tree from the rest, we can do a Taylor expansion of the first term as:
$$
\Delta\mathcal{L}=\frac{1}{N}\sum_{i=1}^{N}[g_if_{k+1}(\boldsymbol{x}_i)+\frac{1}{2}h_if^{2}_{k+1}(\boldsymbol{x}_i)]+\frac{1}{K+1} \Omega(f_{k+1})+(\frac{1}{K+1}-\frac{1}{K})\sum_{k=1}^{K}\Omega(f_k),
$$
where $g_i=\partial_{\hat{y}_i}l$ and $h_i=\partial^{2}_{\hat{y}_i}l$. It can be seen that only the first two terms is related to the $(K+1)$th tree structure. 

Thus the training algorithm for the $(K+1)$th tree is to minimize the first two terms. Expressing $f$ in terms of tree strucuture and leaf, we have
\begin{align}
\delta\mathcal{L}=&\frac{1}{N}\sum_{i=1}^{N}[g_i\omega_{q(\boldsymbol{x}_i)}+\frac{1}{2}h_i\omega^{2}_{q(\boldsymbol{x}_i)}]+\frac{1}{K+1} (\gamma T + \frac{1}{2}\lambda \sum_{t=1}^{T}\omega_{t}^{2})\\
                 =&\sum_{t=1}^{T}[\frac{1}{K+1} \gamma+g_t \omega_{t} + \frac{1}{2}(h_t+\lambda) \omega_{t}^{2}]
\end{align}

where $g_t=\sum_{ \{i|q(\boldsymbol{x}_i)=t\}}\frac{g_i}{N}$ and $h_t=\sum_{ \{i|q(\boldsymbol{x}_i)=t\}}\frac{h_i}{N}$.

The expresssion in the second line can be used to derive a local minimization approach to find the global minization, (however, the global minimization is not guaranteed). The method is that the current leaf is split into left and right leaf to check whether the loss from the sum of the two children leaf is smaller than the loss of current leaf, if true, the current leaf is split otherwise not. Because the above expression does not involve interaction between each leaf, the minimization is given by 
$$
\omega_t=-\frac{g_t}{h_t+\lambda}.
$$
Comment: a properly defined loss function should have both $h_t$ and $\lambda$ positive. And minimal value is given by:
$$
-\frac{g^2_t}{2(h_t+\lambda)}+\frac{\gamma}{K+1}.
$$
So by apliting, the loss function reduction is given by:
$$
\frac{g^2_l}{2(h_l+\lambda)}+\frac{g^2_r}{2(h_r+\lambda)}-\frac{g^2_t}{2(h_t+\lambda)}-\frac{\gamma}{K+1},
$$
which support spliting if smaller than zero other.

## Experiment

In [1]:
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
from my_ml_lib.supervised_learning import tree_based_models as tbm
from my_ml_lib.supervised_learning import xgboost 
from my_ml_lib.supervised_learning import linear_models as lm

In [3]:
redwine=pd.read_csv("../../../data_set/winequality-red.csv")
tables=np.array(redwine.values.tolist(),dtype=np.float32)
X=tables[:,:-1]
Y=tables[:,-1:]
feature_name_dtype=[[redwine.columns[i],'c'] for i in range(len(redwine.columns)-1)]
# cross validation:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [5]:
tree_node=tbm.tree_node_decorator(lm.average_regression,hparameter=0.1)           # this is a class type not class object
decision_tree=tbm.decision_tree_decorator(tree_node,max_depth=5,gamma=0.5,min_sample_size=100)    # this is a class object
xgboost_tree=xgboost.xgboost_regression_decorator(decision_tree,max_num_trees=5)()
xgboost_tree.fit(X_train,Y_train,feature_name_dtype)
np.sqrt(np.mean((xgboost_tree.predict(X_test)-Y_test)**2))/np.mean(Y_test)
xgboost_tree.plot_tree()

0.1438569288800122

## References
<a id="1">[1]</a> 
Tianqi Chen, Carlos Guestrin (2016). 
XGBoost: A Scalable Tree Boosting System. 
ArXiv:1603.02754v3, 10 Jun 2016.